In [1]:
from metro_utils import graph, extract_stations, pairs2rails, subsets, total_distance, gen_distance_matrix
from station import Station
from city import City
from collections import defaultdict as dd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

F mild changes to base classes:

- `graphs(..., fix_aspect=False)` optional arg to stop scaling of the visuals; and addition of low-quality station id labels
- `__repr__` for station is slightly different, was getting confused if I was dealing with 0 or Station 0

Other key points:

- Test cases in json file, I'll add to this but so far the main ones are `hexagon`, `loop_feasibility` and the `_cluster` family. See how-to import in new notebook


# Stage 0
TSP

$$\begin{aligned}
&z=\min \sum_{i=1}^{n} \sum_{j=1}^{n} c_{i j} x_{i j} \\
&\text { s.t. } \\
&\sum_{j=1}^{n} x_{i j}=1, \quad i \in I, \\
&\sum_{i=1}^{n} x_{i j}=1, \quad j \in I, \\
&\sum_{i \in S}\sum_{j \in S} x_{ij} \leq |S|-1, \qquad S \subset I, 2 \leq |S| \leq n-1, \\
&x_{i j} \in\{0,1\}, \quad i, j=1, \ldots, n.
\end{aligned}$$

In [2]:
# Load test cases
# Highly recommend https://www.mathsisfun.com/data/cartesian-coordinates-interactive.html for creating these easily

from json import load

ignore_fields = ["description", "comment", "source"]
def filter_ignore(d):
    return {k: v for k, v in d.items() if k not in ignore_fields}

with open("test_cases.json", "r") as f:
    tests = load(f, object_hook=filter_ignore)

In [3]:
# Choose problem to use in code below
test_choice = "4_cluster"
locations = tests[test_choice]["locations"]
N_STATIONS = len(locations)
kinds = tests[test_choice]["kinds"]

city = City(**tests[test_choice])
stations = city.stations

dist = gen_distance_matrix(city)
S = list(range(N_STATIONS))
subs = subsets(S)

In [ ]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model()

x = m.addVars(N_STATIONS, N_STATIONS, vtype=GRB.INTEGER, name="x")

m.setObjective(gp.quicksum(dist[s1, s2]*x[s1, s2] for s1 in S for s2 in S if s1 != s2), GRB.MINIMIZE)

# leave each station only once
m.addConstrs(gp.quicksum(x[s1, s2] for s1 in S if s1 != s2 ) == 1 for s2 in S)
# enter each station only once
m.addConstrs(gp.quicksum(x[s1, s2] for s2 in S if s1 != s2 ) == 1 for s1 in S)
# Subtour elimination constraints
for sub in subs:
    m.addConstr(gp.quicksum(x[s1, s2] for s1 in sub for s2 in sub if s1 != s2) <= len(sub) - 1)

m.optimize()

In [ ]:
best = pairs2rails([extract_stations(v.varName, city) for v in m.getVars() if v.x > 1e-6])
graph(stations, best, equal_aspect=True)

## Stage 2 - MIP Approach

In [4]:
def demand_by_centrality(stations, C=3):
    """Simple proxy demand for testing the framework
    # d = Cexp(-norm_2(rescaled(s - map_midpoint)))
    # Rescaled(s) = s' is s.t. norm_1(s') <= 1
    # And map_midpoint = midpoint/center of the minimal bounding box for the stations
    #
    # ASSUMES:
    # That more central stations (by how MM generates its worlds) get/need more traffic
    """
    import math
    lb = min([s.x for s in stations])
    rb = max([s.x for s in stations])
    db = min([s.y for s in stations])
    ub = max([s.y for s in stations])

    cx = (lb+rb)/2
    cy = (db+ub)/2
    d = np.zeros([n])
    for i,s in enumerate(stations):
        d[i] = C*math.exp(-math.sqrt(((s.x-cx)/(rb-lb))**2 + ((s.y-cy)/(ub-lb))**2))
    return d

In [5]:
# A few modifications, but otherwise the same hyperparameters
n = N_STATIONS  # no. customers
M = 4
D = range(M) # set of vehicles
V = S  # customers
DUMMY = n  # dummy depot's index
V0 = S + [DUMMY]  # customers + depot

Q = n  # capacities of vehicles (assumed homogenous)
K = 1 # 3
L = 100 # N_STATIONS - (M-1)*K
mk = 1 # Num trains from kth depot

# Slightly larger cost matrix, just with a row/col of zeros for the dummy depot
cost = np.zeros([n+1,n+1])
cost[:n,:n] = dist 

# Demands. Unit demands is TSP-like if capacity Q = |S|
# d = np.ones([n])

# OR assign demand based on centrality. Normalise demand to ensure feasibility
d = demand_by_centrality(stations, C=2)
d = d * Q * M / d.sum()

# Special-case: A `hexagon` demand assignment to showcase overlapping
# d = [0.8, 8, 0.8, 8, 0.8, 0.8, 0.8]

## SDVRP - Based Implementation

*Done, but needs cleaning, and may not be most powerful MIP model possible*

**Origin**: Archetti et al.'s Tabu search paper <a href="https://www.jstor.org/stable/25769283">here</a> though it is expanded on in **The Split Delivery Vehicle Routing Problem: A Survey** (Archetti et al.) <a href=" https://www-or.amp.i.kyoto-u.ac.jp/ramp2006/paper/2-1.pdf">here</a>

### Modifications made to the SDVRP model above:

0. I have been setting 'demands' to be inversely proportional to the 'centre' of the city network; this is to introduce higher-demand cities in an attempt to promote crossings.
1. Added a single 'dummy depot' from which all vehicles must leave and enter. This is to detach from 'specified/fixed depot' concept.
2. Added extra constraints to discourage self-joins (could probably merge into another constraint)
3. Added extra constraints to enforce that all vehicles must leave, and enter, dummy depot exactly once
4. Switched from DFJ SEC's to MTZ SEC's; I feel it is easier to work with when it comes to allowing loops within a route, and furthermore avoids exponentially many constraints
5. Modified MTZ SEC to 'allow' a subtour-forming join IF that node would connect directly to the dummy depot

> (Refresher: MTZ SEC's) $u_i − u_j+ px_{ij} \leq p − 1\quad\forall i,j \in V$ (here $p$ = `N_STATIONS`)

> (What I used) $u_i^{[k]} − u_j^{[k]}+ px_{ij}^{[k]} \leq p − 1 + px_{jD}^{[k]}\quad\forall i,j \in V,\ k\in D$

- Idea: if (i -> j), if j is not the last trainstop prior to a depot, then it's just ordinary MTZ but per-route
- If however (i -> j) and (j -> depot) then (because we'll have u[j,k] = 0 < u[i,k]) we add in p again 

6. Capacities of vehicles/trains are homogenous, but easily modifiable

In [ ]:
import gurobipy as gp
from gurobipy import GRB

m = gp.Model()
MIN_EDGES = 1  # relevant to an extra constraint that avoids short-lived train stops; Ants approach would do better here

x = m.addVars(N_STATIONS+1, N_STATIONS+1, M, vtype=GRB.INTEGER, name="x") # node i --> node j on path of vehicle k
y = m.addVars(N_STATIONS, M, lb=0.0, name="y")  # quantity of ith site's demand satisfied by kth vehicle/route
u = m.addVars(N_STATIONS, M, lb=0.0, name="u")  # MTZ potentials

m.setObjective(gp.quicksum(cost[i,j]*x[i,j,k] for i in V0 for j in V0 for k in D), GRB.MINIMIZE)

# EDGE CONSISTENCY CONSTRAINTS

# 2: at least one edge enters every j (including the depot)
m.addConstrs(gp.quicksum(x[i,j,k] for i in V0 for k in D) >= 1 for j in V0)
# 3: in == out for every node + depot, per-route basis (and hence in general)
m.addConstrs((gp.quicksum(x[i,j,k] for i in V0) - gp.quicksum(x[j,i,k] for i in V0) == 0 for j in V0 for k in D))

# SUBTOUR CONSTRAINTS 
# 4_modified: SEC using MTZ
m.addConstrs((u[i,k] - u[j,k] + N_STATIONS*x[i,j,k] <= N_STATIONS-1 + \
                  N_STATIONS*x[j,DUMMY,k] for i in V for j in V if i != j for k in D))

# DEMAND CONSTRANTS

# 5: ith demand can only be sated by route k if there's an outbound edge from i on that path
m.addConstrs(y[i,k] <= d[i]*gp.quicksum(x[i,j,k] for j in V0) for i in V for k in D)

# 6: total sated demand @ i must meet demand
m.addConstrs(gp.quicksum(y[i,k] for k in D) == d[i] for i in V)

# 7: (MODIFIED FOR HOMOGENOUS CAPACITY) total sated demand by k cannot exceed capacity of vehicle
m.addConstrs((gp.quicksum(y[i,k] for i in V) <= Q for k in D))

# 8,9: Domain constraints (already handled)

# EXTRA constraints

# X1. No self-joins
m.addConstrs(x[i,i,k] <= 0 for i in V0 for k in D)
# X2. For a given k, you must (and only once) enter the dummy depot, and leave the dummy depot
m.addConstrs(gp.quicksum(x[i,DUMMY,k] for i in V) == mk for k in D)
m.addConstrs(gp.quicksum(x[DUMMY,i,k] for i in V) == mk for k in D)
# X3. Perhaps obvious, but the same vehicle cannot cross the same connection twice (excluding depot here is key)
m.addConstrs((x[i,j,k] + x[j,i,k] <= 1 for i in V for j in V for k in D))
# X4. Routes must have more than 2 edges
m.addConstrs((gp.quicksum(x[i,j,k] for i in V0 for j in V0) >= MIN_EDGES for k in D))

# Experimental constraints

# E1. If d() satisfies triangle ineq, then for a pair of routes, can only share one customer (see the Survey paper)
# == NO EDGE SHARING
# m.addConstrs((gp.quicksum(x[i,j,k] for k in D) <= 1 for i in V for j in V))
# E2. (added for negative costs to stop lollygagging) 
# total number of edges should not exceed N_STATIONS + 2*M (to allow loops)
# m.addConstr(gp.quicksum(x[i,j,k] for i in V0 for j in V0 for k in D) <= N_STATIONS + 2*M)

# Special case: Used for testing feasibility of a loop in "loop_feasibility" (3 here is the depot)
#m.addConstr(x[3,2,0] == 1)
#m.addConstr(x[2,1,0] == 1)
#m.addConstr(x[1,0,0] == 1)
#m.addConstr(x[0,2,0] == 1)
#m.addConstr(x[2,3,0] == 1)

m.optimize()

In [ ]:
for var in m.getVars():
    if abs(var.x) > 0 or var.varName[0] in "y":
        print(var.varName, var.x)

In [ ]:
from ast import literal_eval
SOURCE_IDX = 0
TARGET_IDX = 1
RAIL_IDX = 2

# Modified connections definition to disquality report of dummy depot-related edges
connections = [extract_stations(v.varName, city) for v in m.getVars() if v.x > 1e-6 and v.varName[0] == 'x' and \
                  literal_eval(v.varName[1:])[SOURCE_IDX] != DUMMY and literal_eval(v.varName[1:])[TARGET_IDX] != DUMMY]
display(connections)
#connections = [extract_stations(v.varName, city) for v in m.getVars() if v.x > 1e-6 and v.varName[0] == 'x' and \
#                  literal_eval(v.varName[1:])[0] < n]

rail_pairs = dd(list)
for connection in connections:
    rail_pairs[connection[RAIL_IDX]].append(connection[:2])
    
rail_pairs

In [ ]:
# Flawed - cannot handle the below (hexagon-generated with negative costs)
"""
defaultdict(list,
            {0: [[<0>, <3>],
              [<1>, <4>],
              [<2>, <5>],
              [<3>, <6>],
              [<4>, <2>],
              [<5>, <1>],
              [<6>, <4>]]})"""

def modified_pairs2rails(pairs, debug=False):
    starts = [pair[0] for pair in pairs]
    ends   = [pair[1] for pair in pairs]
    
    for end in ends:
        if end not in starts:
            station = end
            break
    else:
        station = starts[0]
        
    if debug: print(station)
    
    # build the rails
    dict_pairs = dict(pairs)
    rev_pairs = {v:k for (k,v) in dict_pairs.items()}
    # rails is a list that encodes the edge sequence
    rails = [station]
    while rev_pairs:
        if debug: print(rev_pairs)
        station = rev_pairs.pop(station)
        rails.append(station)
    
    return rails[::-1]

def foolproof_pairs2rails(pairs, debug=False):
    # cheeky workaround
    from random import shuffle
    for _ in range(10):
        try:
            shuffle(pairs)
            return modified_pairs2rails(pairs, debug)
        except KeyError:
            continue
    else:
        raise KeyError()

In [ ]:
rails = []

for rail, pairs in rail_pairs.items():
    rails.append(foolproof_pairs2rails(pairs, debug=False))

In [ ]:
graph(stations, rails)

## Ant - Based Implementation (Callum WIP)

**Meeting Notes**
- GTSP (and MGTSP) exist (as does (already-multple) GVRP); they force visitation to each cluster however. Since we don't want forced visits, there's no point to this extension.
- However, maybe there is merit in quantifying inter-cluster connections per route. The GVRP formulation described at least in Kara et al. (2012) allows $w_{p\ r}$, which counts ICC's between clusters p and r. One could say we want minimum edge-cost, but as a secondary objective, high ICC counts overall.

In [11]:
# A few modifications, but otherwise the same hyperparameters
N = N_STATIONS  # no. customers
M = 4
D = range(M) # set of vehicles
V = S  # customers
Y = list(range(N,2*N))  # dummy depots, one per station
V0 = V + Y

Q = n  # capacities of vehicles (assumed homogenous)
K = 2 # 3
L = 6 # N_STATIONS - (M-1)*K
mk = 1 # number of trains per route

In [12]:
# Premise: D routes/vehicles, Y depots, V stations (V0 removed)
# Routes are: y1 -> v -> ... -> v -> y2, where L & K constrain the number of v
# This is more general than nonfixed MMTSP as conservation of train entry/exit @ depots is not necessary, only in aggregate

import gurobipy as gp
from gurobipy import GRB

# Redefining parameters
m = gp.Model()

# node i --> node j on path from depot k (nodes include depots)
x = m.addVars(2*N, 2*N, M, vtype=GRB.INTEGER, name="x")
# node i potential, ONLY for actual customers (not dummy) MODIFIED
u = m.addVars(N, M, vtype=GRB.CONTINUOUS, lb=0.0, name="u")

# MODIFIED
m.setObjective(gp.quicksum(dist[i, j]*x[i, j, k] for k in D for j in V for i in V), GRB.MINIMIZE)

# 2 MODIFIED: Per-route, only mk trains leave the depots
m.addConstrs(gp.quicksum(x[y,j,k] for j in V for y in Y) == mk for k in D)
# 3: MODIFIED - Each city needs at least one incoming edge
m.addConstrs(gp.quicksum(x[i, j, k] for i in V0 for k in D) >= 1 for j in V)
# 4: Per-route, per-city, edges in must equal edges out
m.addConstrs(gp.quicksum(x[i, j, k] for i in V0) - gp.quicksum(x[j, i, k] for i in V0) == 0 for k in D for j in V)
# 5: Per-route, must have mk exits to a depot
m.addConstrs(gp.quicksum(x[j,y,k] for y in Y for j in V) == mk for k in D)



# 6: UPPER-BOUND L for u. POTENTIAL CONSTRICTIONS, ESP. ENFORCING L AS A LIMIT.
#m.addConstrs(u[i,k] + (L - 2)*gp.quicksum(x[y, i, k] for y in Y) - 
#              gp.quicksum(x[i, y, k] for y in Y) <= L - 1 for i in V for k in D)
# 7: LOWER-BOUND K for u. READS LIKE FORCING END-OF-LINE STATIONS TO MEET THE LOWER BOUND
#m.addConstrs(u[i,k] + gp.quicksum(x[y, i, k] for y in Y) + \
#             (2 - K)*gp.quicksum(x[i, y, k] for y in Y) >= 2 for i in V for k in D)
# 8 IGNORED - INTERFERES WITH LOOPS IN A 'DEPOT-PER-CITY' MODEL
# m.addConstrs(x[y, i, k] + x[i, y, k] <= 1 for y in Y for i in V for k in D)
# 9: SEC enforcement, L is a nice proxy here for max potential
m.addConstrs(u[i,k] - u[j,k] + L*gp.quicksum(x[i,j,k] for k in D) <= L - 1 + \
                  N*gp.quicksum(x[j,y,k] for y in Y) for i in V for j in V if i != j for k in D)
#ORIGINALISH m.addConstrs(u[i,k] - u[j,k] + L*gp.quicksum(x[i,j,k] for k in D) + \
#              (L - 2)*gp.quicksum(x[j,i,k] for k in D) <= L - 1 + \
#                  N*gp.quicksum(x[j,y,k] for y in Y) for i in V for j in V if i != j for k in D)

# 10 MODIFIED - include depot
m.addConstrs(x[i, i, k] == 0 for i in V0 for k in D)

# EXTRA constraints (particularly given the ghost depots)
# X2. (ALREADY GOT IT HERE)
# X3. Perhaps obvious, but the same vehicle cannot cross the same connection twice (excluding depot here is key)
m.addConstrs((x[i,j,k] + x[j,i,k] <= 1 for i in V for j in V for k in D))
# X4. Routes must have more than K edges
# Note this needs adding even despite ant's K-enforcement as (8) has been excluded
m.addConstrs((gp.quicksum(x[i,j,k] for i in V for j in V) >= K for k in D))
# X5. Stations can only connect to 'their' depot (reduces symmetry)
m.addConstrs(gp.quicksum(x[j,y,k]+x[y,j,k] for k in D for y in Y if y != j+N) == 0 for j in V)
# X6. Routes must have less than L edges
m.addConstrs((gp.quicksum(x[i,j,k] for i in V for j in V) <= L for k in D))

# For `loop_feasibility` loop checking
# m.addConstr(x[2,1,0] == 1)
# m.addConstr(x[1,0,0] == 1)
# m.addConstr(x[0,2,0] == 1)

m.optimize()

# POTENTIALS WITH DEFAULT 1 ARE ANNOYING ... :(

# HEY 8 MIGHT STOP MULTIPLE TRACKS RETURNING AT THE SAME DEPOT. MODIFY SDVRP.

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2788 rows, 5256 columns and 47304 nonzeros
Model fingerprint: 0xb70b948a
Variable types: 72 continuous, 5184 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [2e-01, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 6e+00]
Presolve removed 846 rows and 3816 columns
Presolve time: 0.03s
Presolved: 1942 rows, 1440 columns, 16272 nonzeros
Variable types: 72 continuous, 1368 integer (1368 binary)

Root relaxation: objective 2.934239e+00, 332 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    2.93424    0   82          -    2.93424      -     -    0s
     0     0    2.97925    0   95          -    2.97925      -     -  

In [ ]:
for var in m.getVars():
    if abs(var.x) > 0 or var.varName[0] in "y":
        print(var.varName, var.x)

In [ ]:
from ast import literal_eval
SOURCE_IDX = 0
TARGET_IDX = 1
RAIL_IDX = 2

# NOTE BIG CHANGE - NOT != DUMMY. NOW < N TO FACTOR FOR MANY DUMMIES.

# Modified connections definition to disquality report of dummy depot-related edges
connections = [extract_stations(v.varName, city) for v in m.getVars() if v.x > 1e-6 and v.varName[0] == 'x' and \
                  literal_eval(v.varName[1:])[SOURCE_IDX] < N and literal_eval(v.varName[1:])[TARGET_IDX] < N]
display(connections)
#connections = [extract_stations(v.varName, city) for v in m.getVars() if v.x > 1e-6 and v.varName[0] == 'x' and \
#                  literal_eval(v.varName[1:])[0] < n]

rail_pairs = dd(list)
for connection in connections:
    rail_pairs[connection[RAIL_IDX]].append(connection[:2])
    
rail_pairs

In [ ]:
# Flawed - cannot handle the below (hexagon-generated with negative costs)
"""
defaultdict(list,
            {0: [[<0>, <3>],
              [<1>, <4>],
              [<2>, <5>],
              [<3>, <6>],
              [<4>, <2>],
              [<5>, <1>],
              [<6>, <4>]]})"""

def modified_pairs2rails(pairs, debug=False):           
    starts = [pair[0] for pair in pairs]
    ends   = [pair[1] for pair in pairs]
    
    for end in ends:
        if end not in starts:
            station = end
            break
    else:
        station = starts[0]
        
    if debug: print(station)
    
    # build the rails
    dict_pairs = dict(pairs)
    rev_pairs = {v:k for (k,v) in dict_pairs.items()}
    # rails is a list that encodes the edge sequence
    rails = [station]
    while rev_pairs:
        if debug: print(rev_pairs)
        station = rev_pairs.pop(station)
        rails.append(station)
    
    return rails[::-1]

In [ ]:
rails = []

for rail, pairs in rail_pairs.items():
    rails.append(foolproof_pairs2rails(pairs, debug=False))

In [ ]:
graph(stations, rails)

### (Mostly out-of-context thoughts) 

### Open problems and relevant discussion

Problems:

1. 'as-is' DFJ SEC in Archetti et al. didn't work, in part because you 'want' some of the kinds of subtour behaviour (customer-to-customer) that it is trying to stop.
    - PATH 1 (implemented): Use MTZ SEC's instead
    - PATH 2a: Allow the subtour if the subset exactly matches with a current vehicle route (quantified by `|S| == sum(x[i,j,v] for i in S for j in S) for any v in V`). Be careful to allow 'subtours' that connect to the depot, but not 'subtours' that ignore the depot.
    - PATH 2b: Use disjunctive ideas to deactivate the |S|-1 constraint in such a situation.
        - SOLN IDEA: the constr is `(E(S,v) <= |S| - 1 + 10e6*(E(S,v) == E(v))`
        - OR set `f[s,v]` integer and constrain with `f[s,v] <= E[v] - E[s,v]`


2. Loops, while permissible upon a solution to (P1) are not optimal, as it is always cheaper to just form a straight route and then go back to dummy-depot.
    - NOTE: It is easily verified that loops are now allowed by setting the cost matrix to be 100% negative, encouraging as long a path as it can have (which is `N_STATIONS+1`). Not very tractable for gurobi though


3. Actual optimisation for MM goals (high station diversity, all-types-reachable). I suspect the 'demand satisfaction' idea can effectively be chucked with zero consequences. Heuristic arguments can be used to motivate these issues.


4. My drafted algorithm is slow for 3-cluster (i.e. `N_STATIONS=12`) where the clusters were forced to connect; more constraints may hinder this. Regardless, some optimisations to the solution procedure, perhaps in the Tabu paper or hinted at in the survey, may be important for pushing forward.


5. Ignoring what I've done, there's an awful lot of 'many-paths-work' once you utilise dummy depots. Ant, nonfixed MmTSP, SDVRP, all can work. So it's a matter of choice.
- Caveat I've noticed: Ant can be well-enough adapted to allow loops IF you set up TWO dummy depots... otherwise you have to kill (8). It's been causing problems for me
> Apologies, but while I'm confident the SDVRP-based will not form bad subtours, I can't remember the reasoning

6. If costs satisfy triangle inequality, the SDVRP optimals have one overlapping customer, max. This would allow another constraint, that each edge can only be crossed by one train max (though that violates some play strategy in MM).
    - (but what if d >>>> Q?)
    - I conjecture it is true for MM since d(a,b) - |b-a| is upper-bounded at ~9% of ||b-a|| and any 3 points will have a corresponding 'convex' path

Random:

- Allowing boosted train capacity (since carriages possible)
- `hexagon` + biased demand weights == easy check that overlap is okay
- `loop_feasibility` + negative costs == easy check that loops are okay 
- `donut` == easy check for a problem that needs a loop solution

# Stage 2 - Brute-Force

In [ ]:
import networkx as nx

def cost(rails):
    # Find distance between all pairs of stations
    G=nx.Graph()

    elist = []
    for rail in rails:
        start = rail[0]
        for end in rail[1:]:
            elist.append((start, end, dist[start.id, end.id]))
            start = end    

    G.add_weighted_edges_from(elist)
    all_paths = list(nx.all_pairs_shortest_path(G))
    
    
    station_dists = []
    all_kinds = {s.kind for s in stations}

    # For each starting station
    for start, paths in all_paths:

        distances = {kind: 1e12 for kind in all_kinds}

        # Get its kind
        for end, path in paths.items():
            # For each other station: If kind is different
            if start.kind != end.kind:
                # Work out distance to that station and retain if min
                distances[end.kind] = min(distances[end.kind], total_distance(path))

        station_dists.append(sum(distances.values()))

    return sum(station_dists)

In [ ]:
from math import factorial
from itertools import permutations
from tqdm import tqdm

def brute_force_tsp(stations):
    max_dist = 1e15
    best = None

    for perm in tqdm(permutations(stations), total=factorial(N_STATIONS)):
        perm = list(perm)
        perm.append(perm[0])
        perm = [perm]
        
        dist = cost(perm)
        if dist < max_dist:
            max_dist = dist
            best = perm
            
    return best

a = brute_force_tsp(stations)

In [ ]:
a

In [ ]:
graph(stations, a)

In [ ]:
from math import factorial

def brute_force_tsp(stations):
    ''' This doesn't allow for loops or overlapping sections but it should '''
    max_dist = 1e15
    best = None

    for perm in tqdm(permutations(stations), total=factorial(N_STATIONS)):
        perm = list(perm)
        perm.append(perm[0])
        for split in range(2, len(perm)):
            split = [perm[:split], perm[split:]] 

            dist = cost(split)
            if dist < max_dist:
                max_dist = dist
                best = split
            
    return best

a = brute_force_tsp(stations)

In [ ]:
a

In [ ]:
graph(stations, a)